### Dota Hero Grid Editor

Hero grids are stored in the following `.json` file in the steam folder:\
`C:\Program Files (x86)\Steam\userdata\YOUR_FRIEND_ID\570\remote\cfg\hero_grid_config.json`

Let's copy it into 
* our working directory - as **a dump file** to play with
* subfolder `./saved` of it^ to keep as **a backup file** in case we do something illegal

In [1]:
# pretty self-explanatory but dota_utils give functions 
# to get hero names by their ids and vise versa
from dota_utils import id_by_name, name_by_id

import json
import shutil
from config import DOTA_FRIENDID

steam_cfg_loc = f'C:\\Program Files (x86)\\Steam\\userdata\\{DOTA_FRIENDID}\\570\\remote\\cfg'
json_rel_loc = '\\hero_grid_config.json'  # rel- relative  
src_steam_cfg = f'{steam_cfg_loc}{json_rel_loc}'  # source
dst_dump = '.'  # destination for dump
dst_backup = './saved'  # destination for backup
shutil.copy2(src_steam_cfg, dst_dump)
_ = shutil.copy2(src_steam_cfg, dst_backup) # _ = just so it doesnt print into output

### READ WRITE CELLS

In [2]:
# READ FROM DUMP FILE
with open('hero_grid_config.json') as json_file:
    data = json.load(json_file)

def write_and_copy():
    # WRITE INTO DUMP FILE 
    with open('hero_grid_config.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    # COPY DUMP FILE BACK TO STEAM_CFG_LOC FOLDER WHEN WE ARE DONE
    src_dump = dst_dump + json_rel_loc
    shutil.copy2(src_dump, steam_cfg_loc)

### Info to plan the Hero Grid
`data` - is the dictionary that is gonna be dumped into the json file back. 

```python
MAX_X = 1200
MAX_Y = 598
```
`data['configs'][0]` is my main Draft grid\
`data['configs'][1]` is Dota Plus levels grid

In [3]:
from typing import Dict, NamedTuple 

MAX_X, MAX_Y = 1200, 598

class Position(NamedTuple):
    x: int | float
    y: int | float
    w: int | float
    h: int | float

class Grid:
    def __init__(self, config_index: int, categories: Dict[str, Position]) -> None:
        self.config_index: int = config_index
        self.categories: Dict[str, Position] = categories

    def update_categories(self):
        for category in data['configs'][self.config_index]['categories']:
            try:
                pos = self.categories[category['category_name']]
            except KeyError as error:
                raise KeyError(
                    f'Category with name "{error}" does not exist in this hero grid.'
                    'Please add this category into your actual grid in Dota 2 client or in file yourself.'
                )
            category['x_position'] = pos.x
            category['y_position'] = pos.y
            category['width'] = pos.w
            category['height'] = pos.h

### Dota Plus Levels Grid

In [4]:
# changeable constants
left_limit_x = 400  # the line between my left and right grid parts
height = 100 
delta = 20
grind_h = 80  # the very last small row

class DotaPlusGrid(Grid):
    def __init__(self) -> None:
        super().__init__(
            config_index=1, 
            categories={  # match these names with the ones you have in the hero grid 
                'Grandmaster': Position(x=0,  y=0, w=left_limit_x/2, h=height),
                'Master': Position(x=left_limit_x/2, y=0, w=left_limit_x/2, h=height),
                'Platinum': Position(x=0, y=height+delta, w=left_limit_x, h=height),
                'Gold': Position(x=0, y=2*(height+delta), w=left_limit_x, h=height),
                'Silver': Position(x=0, y=3*(height+delta), w=left_limit_x, h=MAX_Y - 3*(height+delta)),
                'Bronze': Position(x=left_limit_x, y=0, w=MAX_X-left_limit_x, h=MAX_Y-delta-grind_h),
                'Grind': Position(x=left_limit_x, y=MAX_Y-grind_h, w=MAX_X-left_limit_x, h=grind_h)
            }
    )

    def sort_by_dota_plus_xp(self):
        # todo: hmm, idk how to do it considering 
        # I have data private
        # there is a way from Stratz to get it though
        pass

dota_plus_grid = DotaPlusGrid()
dota_plus_grid.update_categories()
write_and_copy()

In [5]:
data['configs'][0]['categories'][-1]

{'category_name': 'Grind/Arcanas/D+/Cavern/Styles/Missions',
 'x_position': 349.0,
 'y_position': 523,
 'width': 425.5,
 'height': 75,
 'hero_ids': [21, 39, 44, 46]}

### My default picking screen Grid

In [6]:
left_limit_x = 6 * 58 + 1  # 6 for six heroes in pos 1

# positions
height = 100
rows = 5  # 5 positions in dota 
delta_pos = (MAX_Y - height * rows ) / (rows - 1)

# attributes
delta_attr = 10

# the last row for bans and hero recommendations
ban_height = 75
ban_spacing = -5  # space between attributes and bans

class DefaultRolesGrid(Grid):
    def __init__(self) -> None:
        super().__init__(
            config_index=0,
            categories={
                f'pos{i+1}': Position(
                    x=0, 
                    y=0+(height+delta_pos)*i, 
                    w=left_limit_x, 
                    h=height
                )
                for i in range(rows)
            } | {
                name: Position(
                    x=left_limit_x, 
                    y=i*(MAX_Y-ban_height-ban_spacing)/3, 
                    w=MAX_X-left_limit_x, 
                    h=(MAX_Y-ban_height-ban_spacing-2*delta_attr)/3
                )
                for i, name in enumerate(['Str', 'Agi', 'Int'])
            } | {
                name: Position(
                    x=left_limit_x+i*(MAX_X-left_limit_x)/2, 
                    y=MAX_Y-ban_height, 
                    w=(MAX_X-left_limit_x)/2, 
                    h=ban_height
                )
                for i, name in enumerate(['Grind/Arcanas/D+/Cavern/Styles/Missions', 'bans'])
            }
        )

    def sort_attr_categories_by_name(self):
        for category in data['configs'][self.config_index]['categories']:
            if category['category_name'] in ['Str', 'Agi', 'Int']:
                hero_names = [name_by_id(i) for i in category['hero_ids']]
                new_names = sorted(hero_names, key=str.casefold)

                new_ids = [id_by_name(n) for n in new_names]
                category['hero_ids'] = new_ids


default_grid = DefaultRolesGrid()
default_grid.update_categories()
default_grid.sort_attr_categories_by_name()
write_and_copy()